In [1]:
import pandas as pd
import numpy as np

data = {
    "name": ["apple", "banana", "car", "bus", "bike", "grapes", "truck"],
    "type": ["fruit", "fruit", "vehicle", "vehicle", "vehicle", "fruit", "vehicle"]
}
df = pd.DataFrame(data)
df["embedding"] = df["name"].apply(lambda _: np.random.rand(4).tolist())


In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

client.recreate_collection(
    collection_name="hybrid_search_demo",
    vectors_config={"size": 4, "distance": "Cosine"}
)

client.upload_collection(
    collection_name="hybrid_search_demo",
    vectors=df["embedding"].tolist(),
    payload=df.to_dict(orient="records"),
    ids=list(range(len(df)))
)


/tmp/ipykernel_162703/3654380714.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [4]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue

query_vector = np.random.rand(4).tolist()

results = client.search(
    collection_name="hybrid_search_demo",
    query_vector=query_vector,
    limit=5,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="type",
                match=MatchValue(value="vehicle")
            )
        ]
    )
)

for hit in results:
    print(f"{hit.payload['name']} | Type: {hit.payload['type']} | Score: {hit.score:.4f}")


/tmp/ipykernel_162703/1027840466.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


truck | Type: vehicle | Score: 0.7888
bike | Type: vehicle | Score: 0.7569
car | Type: vehicle | Score: 0.6767
bus | Type: vehicle | Score: 0.4755
